In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Lenovo\\Documents\\Chicken_DiseseClassification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Lenovo\\Documents\\Chicken_DiseseClassification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool    
    params_weights: str
    
    params_classes: int
        
        

In [6]:
from src.CNNClassifier import logger
from src.CNNClassifier.constants import *
from src.CNNClassifier.utils.common import read_yaml, create_directories

Box is working ✅


In [12]:
class ConfigurationManager:
    def __init__(self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([Path(self.config.artifacts_root)])
        
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])
        
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )
        
        logger.info(f"PrepareBaseModelConfig: {prepare_base_model_config}")
        return prepare_base_model_config

In [13]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [14]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top)

        self.save_model(self.config.base_model_path, self.model)
        logger.info(f"Base model is saved at {self.config.base_model_path} successfully.")  

    @staticmethod
    def __prepare_full_model(model, classes,freeze_all, freeze_till,learning_rate):
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False

        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax")(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction)

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )
        return full_model

    def update_base_model(self):
        self.full_model = self.__prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        self.save_model(self.config.updated_base_model_path, self.full_model)
        logger.info(f"Updated base model is saved at {self.config.updated_base_model_path}")

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


        

In [16]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()

    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)

    prepare_base_model.get_base_model()

    prepare_base_model.update_base_model()

except Exception as e:
    logger.exception(e)
    raise e    


[2025-09-28 19:08:47,752 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-09-28 19:08:47,766 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-09-28 19:08:47,770 : INFO : common : Created directory at: src\artifacts]
[2025-09-28 19:08:47,775 : INFO : common : Created directory at: src/artifacts/prepare_base_model]
[2025-09-28 19:08:47,777 : INFO : 1559359162 : PrepareBaseModelConfig: PrepareBaseModelConfig(root_dir=WindowsPath('src/artifacts/prepare_base_model'), base_model_path=WindowsPath('src/artifacts/prepare_base_model/base_model.h5'), updated_base_model_path=WindowsPath('src/artifacts/prepare_base_model/updated_base_model.h5'), params_image_size=BoxList([224, 224, 3]), params_learning_rate=0.01, params_include_top=False, params_weights='imagenet', params_classes=2)]
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 10s 0us/step
[2025-09-28 19:08:58,885 : WARNING : saving_api : You are saving your model as an HDF5 file via `model.save()` or `keras.